# Preparation

## Authentication

Before requests to orchestration can be issued, we need to provide authentication details to the SDK. This can be done either via a configuration file or via the environment. Make sure to check out the [SAP generative AI hub SDK project description](https://pypi.org/project/generative-ai-hub-sdk/) for more details. Below you will find an example for authenticating via environment variables using this very notebook.

> **WARNING:**
> Below code should never be used in production scenarios and is only for the purpose of illustrating which environment variables to use!
> Credentials should never be defined in code!

In [ ]:
import os

os.environ["AICORE_AUTH_URL"] = "redacted"
os.environ["AICORE_BASE_URL"] = "redacted"
os.environ["AICORE_CLIENT_ID"] = "redacted"
os.environ["AICORE_CLIENT_SECRET"] = "redacted"
os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"] = "redacted"
os.environ["AICORE_RESOURCE_GROUP"] = "redacted"

You will need to authenticate in every exercise.

## Intializing the Orchestration Service

Typically, a virtual deployment of Orchestration must be configured before any interactions can occur. Once deployed, you will have access to a unique endpoint URL. For this session, the deployment has already been created for you. The corresponding URL, along with the authentication credentials, have been provided to you in advance. If you've adhered to the prerequisite guidelines, the URL should already be present in the environment variable `AICORE_ORCHESTRATION_DEPLOYMENT_URL`.

# Templating

Now that everything is prepared, we can write our first basic orchestration pipeline. The first fundamental module we will look at is templating. The templating module provides capabilities to define prompt skeletons that can then be parameterized per inference call. To check out how this works we first up select a Large Language Model (LLM) that will be used for inference.

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(
    name="gemini-1.5-flash",
    version="latest",
    parameters={"max_tokens": 256, "temperature": 0.2},
)

Now we can create a template using the template object provided by the SAP generative AI hub SDK.

In [ ]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage(
            "Translate the following text to {{?to_lang}}: {{?text}}",
        ),
    ],
    defaults=[
        TemplateValue(name="to_lang", value="English"),
    ],
)

The code above creates a template that provides

- a system message,
- a user message that leverages templating syntax,
- default values for the introduced template parameters.

Currently there are three message types available:
- `SystemMessage`: A message for priming AI behavior. The system message is usually passed in as the first of a sequence of input messages.
- `UserMessage`: A message from a user.
- `AssistantMessage`: A message of the LLM.

Parameters are defined within the message string using the following syntax: `{{?param_name}}`.

Next up we create a orchestration configuration from the created objects.

In [ ]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

Lastly, we can call the orchestration service. Note that the actual template values are now passed to the `run` method. The `TemplateValue` name parameter corresponds to the parameter name `text` provided in the user message string. The parameter `to_lang` is omitted and the default defined in the PromptTemplate is used.

In [ ]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService(
    api_url=os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"],
    config=config,
)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="Interaktives Lernen mit SAP.",
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

# Model Harmonization

Orchestration harmonizes model usage, removing the need for prompting each model in vendor specific fashion. You can easily switch between a variety of models. Check out [this SAP Note](https://me.sap.com/notes/3437766) for further information regarding model availability. For this Jump-Start session model access is restricted to `gemini-1.5-flash` and `meta--llama3-70b-instruct` though. The code below will demonstrate how to easily switch between models, based on the templating code above!

In [ ]:
orchestration_service.config.llm = LLM(
    name="meta--llama3-70b-instruct",
)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="Interaktives Lernen mit SAP.",
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

You can switch between those two models and compare their responses throughout all exercises. Simply change the `name` parameter of the LLM module configuration.

# Streaming

When generating larger amounts of text you will notice that the time it takes the LLM to respond will increase.
To make your application feel more responsive you can leverage the SDKs streaming capabilities where supported. 
When using this option you will get portions of the LLM answer as the text gets generated.

In [ ]:
# call without streaming
orchestration_service.config.llm = LLM(
    name="gpt-4o-mini",
)

result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="text",
            value="""SAP Geschichte: Im Jahre 1972 hatten fünf Unternehmer eine Vision für das Geschäftspotenzial betriebswirtschaftlicher Software.
Mit ihren Lösungen SAP R/2 und SAP R/3 etablierte SAP einen weltweiten Standard für ERP-Software (Enterprise Resource Planning).
Mit SAP S/4HANA folgte dann die nächste Generation der SAP-ERP-Software.
Die Plattform macht die In-Memory-Technologie für ERP-Anwender nutzbar, sodass riesige Datenmengen in Echtzeit verarbeitet werden können.
Zudem lassen sich neue Technologien wie künstliche Intelligenz (KI) und maschinelles Lernen einbinden.
Mit einem Kunden und einer Handvoll Mitarbeiter schlug die SAP einen Weg ein, der nicht nur die Welt der IT, sondern auch die Art und Weise, wie Unternehmen ihre Geschäfte abwickeln, langfristig verändern sollte.
Nun, mit mehr als 50 Jahren Erfahrung und mehr als 400.000 Kunden, ist die SAP mehr denn je von dem Pioniergeist beflügelt, der ihre Gründer inspiriert hat.

Die Anfangsjahre
Am 1. April 1972 gründen fünf ehemalige IBM-Mitarbeiter das Unternehmen „Systemanalyse Programmentwicklung“.
Die Vision von Dietmar Hopp, Hasso Plattner, Claus Wellenreuther, Klaus Tschira und Hans-Werner Hector: Standardsoftware für Unternehmen zu entwickeln, die alle betrieblichen Abläufe integriert und es ermöglicht, Daten in Echtzeit zu verarbeiten.
Die SAP-Gründer und ihre Mitarbeiter entwickeln die Software im engen Austausch mit den Mitarbeitenden der Kunden. 
Bei Unternehmen wie ICI, Knoll, Burda und Linde entstehen bis 1975 in enger Zusammenarbeit mit deren Mitarbeitern Programme für die Finanzbuchhaltung (RF), die Rechnungsprüfung und die Materialwirtschaft (RM).
Neben der Echtzeitverarbeitung der Daten setzen die SAP-Gründer auch auf Standardisierung und Integration.
Diese Kombination bildet die Grundlage für den Aufstieg der SAP zum Weltmarktführer für betriebswirtschaftliche Software.
Schon 1979 beginnt die Entwicklung der zweiten Softwaregeneration der SAP.
1980 ziehen die rund 80 Mitarbeiter ins erste eigene Bürogebäude in Walldorf.

Von R/3 zum globalen Lösungsanbieter
Schon 1987 – während SAP R/2 boomt und ein Jahr vor dem Börsengang – beschließen die SAP-Chefs, die dritte Generation als multiplattformfähige Software zu entwickeln.
1992 beginnt der Siegeszug der Client-Server-Software SAP R/3, die zu einem Wegbereiter der globalisierten Wirtschaft wird und SAP auch selbst endgültig zu einem Global Player mit Entwicklungszentren in aller Welt macht.
1999 kündigt die SAP als Antwort auf Internet und New Economy die mySAP.com-Strategie an.
Zehn Jahre später beschließt das Unternehmen, sich neben dem ERP-Markt auf drei weitere Zukunftsfelder zu konzentrieren: Mobil-, Datenbanktechnologie und die Cloud. Um möglichst schnell zu einem führenden Unternehmen auf diesen Märkten zu werden, übernimmt SAP mehrere Konkurrenten, darunter Business Objects, Sybase, Ariba, SuccessFactors, Fieldglass und Concur.

Mit SAP HANA in die Cloud
Von 2011 an setzen erste Kunden die In-Memory-Datenbank SAP HANA ein und analysieren damit Daten in Sekunden, wofür sie zuvor Tage und Wochen brauchten. Vier Jahre später stellt SAP ihre neueste Generation der Unternehmenssoftware vor, die vollständig auf SAP HANA basiert: SAP S/4HANA.
SAP unterstützt Kunden heute auf dem Weg zum nachhaltigen, vernetzten und intelligenten Unternehmen, indem sie Lösungen, Technologie und Best Practices verknüpft, die für integrierte, digitale Geschäftsprozesse in der Cloud benötigt werden.
SAP bietet Wahlfreiheit zwischen den vier größten Public-Cloud-Plattform-Betreibern.
Die SAP Business Technology Platform unterstützt Kunden auf dem Weg zum intelligenten Unternehmen, egal ob Cloud-, On-Premise- oder hybride IT-Landschaft, und ist ein wesentliches Element des SAP-Angebots „RISE with SAP“. 
SAP zählt derzeit rund 300 Millionen Cloud-Anwender und bietet mit mehr als 100 Lösungen für alle Unternehmensfunktionen das größte Cloud-Portfolio aller Anbieter. SAP betreibt 47 Rechenzentren an 27 Standorten in 15 Ländern.
""",
        )
    ]
)

print(result.orchestration_result.choices[0].message.content)

In [ ]:
# call with streaming
orchestration_service.config.llm = LLM(
    name="gpt-4o-mini",
)

response = orchestration_service.stream(
    template_values=[
        TemplateValue(
            name="text",
            value="""SAP Geschichte: Im Jahre 1972 hatten fünf Unternehmer eine Vision für das Geschäftspotenzial betriebswirtschaftlicher Software.
Mit ihren Lösungen SAP R/2 und SAP R/3 etablierte SAP einen weltweiten Standard für ERP-Software (Enterprise Resource Planning).
Mit SAP S/4HANA folgte dann die nächste Generation der SAP-ERP-Software.
Die Plattform macht die In-Memory-Technologie für ERP-Anwender nutzbar, sodass riesige Datenmengen in Echtzeit verarbeitet werden können.
Zudem lassen sich neue Technologien wie künstliche Intelligenz (KI) und maschinelles Lernen einbinden.
Mit einem Kunden und einer Handvoll Mitarbeiter schlug die SAP einen Weg ein, der nicht nur die Welt der IT, sondern auch die Art und Weise, wie Unternehmen ihre Geschäfte abwickeln, langfristig verändern sollte.
Nun, mit mehr als 50 Jahren Erfahrung und mehr als 400.000 Kunden, ist die SAP mehr denn je von dem Pioniergeist beflügelt, der ihre Gründer inspiriert hat.

Die Anfangsjahre
Am 1. April 1972 gründen fünf ehemalige IBM-Mitarbeiter das Unternehmen „Systemanalyse Programmentwicklung“.
Die Vision von Dietmar Hopp, Hasso Plattner, Claus Wellenreuther, Klaus Tschira und Hans-Werner Hector: Standardsoftware für Unternehmen zu entwickeln, die alle betrieblichen Abläufe integriert und es ermöglicht, Daten in Echtzeit zu verarbeiten.
Die SAP-Gründer und ihre Mitarbeiter entwickeln die Software im engen Austausch mit den Mitarbeitenden der Kunden. 
Bei Unternehmen wie ICI, Knoll, Burda und Linde entstehen bis 1975 in enger Zusammenarbeit mit deren Mitarbeitern Programme für die Finanzbuchhaltung (RF), die Rechnungsprüfung und die Materialwirtschaft (RM).
Neben der Echtzeitverarbeitung der Daten setzen die SAP-Gründer auch auf Standardisierung und Integration.
Diese Kombination bildet die Grundlage für den Aufstieg der SAP zum Weltmarktführer für betriebswirtschaftliche Software.
Schon 1979 beginnt die Entwicklung der zweiten Softwaregeneration der SAP.
1980 ziehen die rund 80 Mitarbeiter ins erste eigene Bürogebäude in Walldorf.

Von R/3 zum globalen Lösungsanbieter
Schon 1987 – während SAP R/2 boomt und ein Jahr vor dem Börsengang – beschließen die SAP-Chefs, die dritte Generation als multiplattformfähige Software zu entwickeln.
1992 beginnt der Siegeszug der Client-Server-Software SAP R/3, die zu einem Wegbereiter der globalisierten Wirtschaft wird und SAP auch selbst endgültig zu einem Global Player mit Entwicklungszentren in aller Welt macht.
1999 kündigt die SAP als Antwort auf Internet und New Economy die mySAP.com-Strategie an.
Zehn Jahre später beschließt das Unternehmen, sich neben dem ERP-Markt auf drei weitere Zukunftsfelder zu konzentrieren: Mobil-, Datenbanktechnologie und die Cloud. Um möglichst schnell zu einem führenden Unternehmen auf diesen Märkten zu werden, übernimmt SAP mehrere Konkurrenten, darunter Business Objects, Sybase, Ariba, SuccessFactors, Fieldglass und Concur.

Mit SAP HANA in die Cloud
Von 2011 an setzen erste Kunden die In-Memory-Datenbank SAP HANA ein und analysieren damit Daten in Sekunden, wofür sie zuvor Tage und Wochen brauchten. Vier Jahre später stellt SAP ihre neueste Generation der Unternehmenssoftware vor, die vollständig auf SAP HANA basiert: SAP S/4HANA.
SAP unterstützt Kunden heute auf dem Weg zum nachhaltigen, vernetzten und intelligenten Unternehmen, indem sie Lösungen, Technologie und Best Practices verknüpft, die für integrierte, digitale Geschäftsprozesse in der Cloud benötigt werden.
SAP bietet Wahlfreiheit zwischen den vier größten Public-Cloud-Plattform-Betreibern.
Die SAP Business Technology Platform unterstützt Kunden auf dem Weg zum intelligenten Unternehmen, egal ob Cloud-, On-Premise- oder hybride IT-Landschaft, und ist ein wesentliches Element des SAP-Angebots „RISE with SAP“. 
SAP zählt derzeit rund 300 Millionen Cloud-Anwender und bietet mit mehr als 100 Lösungen für alle Unternehmensfunktionen das größte Cloud-Portfolio aller Anbieter. SAP betreibt 47 Rechenzentren an 27 Standorten in 15 Ländern.
""",
        )
    ],
    stream_options={
        'chunk_size': 1
    }
)

for chunk in response:
    print(chunk.orchestration_result.choices[0].delta.content, end='')

In the following exercises we will use streaming where possible.

# Summary

Within this exercise you learned how to create a basic Orchestration pipeline that uses the Templating module. Also, you changed the model used for inference with ease. Let's explore more modules in the following exercises. Continue to [Exercise 2 - Orchestration Content Filtering](./ex2.ipynb).